In [43]:
######################## Cleaning The Tweets ##########################
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
filename = 'train.csv'
df = pd.read_csv(filename)
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

df['tweet'] = df['tweet'].apply(lambda x: p.clean(re.sub('#', '',x))) #Tweet Cleaner
df['tweet'] = df['tweet'].apply(lambda x: x.lower()) #Lower Casing
df['tweet'] = df['tweet'].str.replace('[^\w\s]','') #Removing Punctuations
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))#STopwords Removal
df['tweet'] = df['tweet'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

df.to_csv('clean_'+str(filename))

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import preprocessor as p
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('clean_train.csv')
df.head()

,id,label,tweet
0,1,0,father dysfunctional selfish drag kid dysfunct...
1,2,0,thanks lyft credit cant use cause dont offer w...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation


In [3]:
df = df.dropna()
#normal = df[df['label']==0]
#hate = df[df['label']==1]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000,ngram_range=(1,2), min_df=5, max_df=0.5, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(df['tweet']).toarray()

In [11]:
X.shape

(31931, 2000)

In [12]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.3, random_state=0)

In [18]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictions = gnb.predict(X_test)
#################### Gaussian NB Result #########################
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
#print(accuracy_score(y_test, predictions))
print('Training Classification Report\n',classification_report(y_train,gnb.predict(X_train)))

[[6098 2786]
 [ 126  570]]
              precision    recall  f1-score   support

           0       0.98      0.69      0.81      8884
           1       0.17      0.82      0.28       696

    accuracy                           0.70      9580
   macro avg       0.57      0.75      0.54      9580
weighted avg       0.92      0.70      0.77      9580

Training Classification Report
               precision    recall  f1-score   support

           0       1.00      0.68      0.81     20808
           1       0.19      1.00      0.32      1543

    accuracy                           0.70     22351
   macro avg       0.59      0.84      0.56     22351
weighted avg       0.94      0.70      0.78     22351



In [17]:
from sklearn.ensemble import RandomForestClassifier
#text_classifier = RandomForestClassifier(n_estimators=30, random_state=0)  
#text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
#print(accuracy_score(y_test, predictions))
print('Training Classification Report\n',classification_report(y_train,text_classifier.predict(X_train)))

[[8788   96]
 [ 353  343]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      8884
           1       0.78      0.49      0.60       696

    accuracy                           0.95      9580
   macro avg       0.87      0.74      0.79      9580
weighted avg       0.95      0.95      0.95      9580

Training Classification Report
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     20808
           1       0.99      0.94      0.96      1543

    accuracy                           0.99     22351
   macro avg       0.99      0.97      0.98     22351
weighted avg       0.99      0.99      0.99     22351



<h1> SMOTE<br></h1>

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import preprocessor as p
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
df = pd.read_csv('clean_train.csv')
df = df.dropna()

In [3]:
data = df.sample(frac=0.4)
len(data)

12772

In [33]:
from sklearn.model_selection import train_test_split
X, X_test, Y, y_test = train_test_split(data['tweet'], data['label'], test_size=0.30, random_state=42)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=3000,ngram_range=(1,2), min_df=5, max_df=0.5, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(X).toarray()
X.shape

(8940, 2586)

In [35]:
X_test = tfidfconverter.transform(X_test).toarray()
X_test.shape

(3832, 2586)

In [36]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2,sampling_strategy=1)
X,Y  = sm.fit_resample(X, Y)
print('After OverSampling, the shape of train_X: {}'.format(X.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(Y.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(Y == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(Y == 0)))

After OverSampling, the shape of train_X: (16654, 2586)
After OverSampling, the shape of train_y: (16654,) 

After OverSampling, counts of label '1': 8327
After OverSampling, counts of label '0': 8327


In [37]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

gnb = GaussianNB()
gnb.fit(X, Y)
predictions = gnb.predict(X_test)
#################### Gaussian NB Result #########################
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
#print(accuracy_score(y_test, predictions))
print('Training Classification Report\n',classification_report(Y,gnb.predict(X)))

[[2919  637]
 [ 111  165]]
              precision    recall  f1-score   support

           0       0.96      0.82      0.89      3556
           1       0.21      0.60      0.31       276

    accuracy                           0.80      3832
   macro avg       0.58      0.71      0.60      3832
weighted avg       0.91      0.80      0.84      3832

Training Classification Report
               precision    recall  f1-score   support

           0       1.00      0.85      0.92      8327
           1       0.87      1.00      0.93      8327

    accuracy                           0.92     16654
   macro avg       0.93      0.92      0.92     16654
weighted avg       0.93      0.92      0.92     16654



In [42]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X, Y)

predictions = text_classifier.predict(X_test)
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
#print(accuracy_score(y_test, predictions))
print('Training Classification Report\n',classification_report(Y,text_classifier.predict(X)))

[[3401  155]
 [ 113  163]]
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      3556
           1       0.51      0.59      0.55       276

    accuracy                           0.93      3832
   macro avg       0.74      0.77      0.76      3832
weighted avg       0.94      0.93      0.93      3832

Training Classification Report
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8327
           1       1.00      1.00      1.00      8327

    accuracy                           1.00     16654
   macro avg       1.00      1.00      1.00     16654
weighted avg       1.00      1.00      1.00     16654



In [39]:
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
Dense(input_dim = 2586, units = 64, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dropout(0.5),
Dense(units = 32, activation = 'relu'),
Dense(units = 16, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 64)                165568    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_13 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 178,561
Trainable params: 178,561
Non-trainable params: 0
________________________________________________

In [40]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X, Y, batch_size = 15, epochs = 5,validation_data=(X_test, y_test))

Epoch 1/5
1111/1111 [==============================] - 2s 1ms/step - loss: 0.3148 - accuracy: 0.8678 - val_loss: 0.3338 - val_accuracy: 0.9368
Epoch 2/5
1111/1111 [==============================] - 1s 1ms/step - loss: 0.0381 - accuracy: 0.9883 - val_loss: 0.3946 - val_accuracy: 0.9327
Epoch 3/5
1111/1111 [==============================] - 1s 1ms/step - loss: 0.0235 - accuracy: 0.9926 - val_loss: 0.4495 - val_accuracy: 0.9311
Epoch 4/5
1111/1111 [==============================] - 1s 1ms/step - loss: 0.0185 - accuracy: 0.9927 - val_loss: 0.4737 - val_accuracy: 0.9293
Epoch 5/5
1111/1111 [==============================] - 1s 1ms/step - loss: 0.0166 - accuracy: 0.9931 - val_loss: 0.4790 - val_accuracy: 0.9418


In [41]:
predictions = model.predict_classes(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[3478   78]
 [ 145  131]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      3556
           1       0.63      0.47      0.54       276

    accuracy                           0.94      3832
   macro avg       0.79      0.73      0.75      3832
weighted avg       0.94      0.94      0.94      3832

0.9418058455114823


c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
